# Comprehensive strategy

## Original prompts

### No think label


In [4]:
def get_nothink_comprehensive_prompt(text):
    """
    Generates a comprehensive prompt for multi-complication assessment, specifically designed *without* a Chain of Thought (CoT) requirement.

    This function creates a "comprehensive strategy" prompt that instructs the AI to evaluate all possible
    post-operative complications simultaneously. Unlike CoT-based prompts, it does not require the AI to
    provide explicit step-by-step reasoning. It directly formats patient medical records and demands
    structured JSON output for diagnosis.

    Key features of this comprehensive strategy prompt (without CoT):
    - Comprehensive evaluation of all 22 predefined complications at once.
    - No mandatory "think" field, allowing for direct diagnosis without explicit step-by-step reasoning.
    - Structured JSON output with diagnosis components.
    - Direct formatting of medical records into the prompt.

    Args:
        text (str): The full content of the prompt definition file (e.g., prompt.md),
                    containing the definitions of post-operative complication categories
                    and their grading rules.

    Returns:
        str: A complete prompt string ready for use with medical record content appended.
             The prompt includes instructions, complication definitions, and the required output format.
    """
    # Define markers to identify and extract specific sections from the input text.
    complication_flag = "## 术后并发症类别"
    grading_flag = "## 常规分级"

    # Extract the section detailing complication categories and their definitions.
    complications = text[text.find(complication_flag) + len(complication_flag): text.find(grading_flag)]
    # Extract the section defining general grading rules and format it for inclusion in the prompt.
    grading = text[text.find(grading_flag) + len(grading_flag):].replace("\n", "\t")

    # Define the initial instruction for the AI model, setting its role and task.
    instruction = '你是一名资深外科医生，你的任务是根据患者的资料，判断患者术后出现哪些并发症，并发症的诊断依据:\n'                
    
    # Define the desired output structure in JSON format, including specific instructions
    # on naming, grading, and handling cases with no complications.
    output_structure = f"""
### 输出json格式
输出一个列表，每个元素为一个字典，包含：
    * name: 并发症的名称，必须是22个预定义并发症之一
    * grading: 严重程度分级，只能是以下四个选项之一：
        * "轻" - 轻度：无需治疗
        * "中" - 中度：需治疗，无长期影响
        * "重" - 重度：导致器官功能受损或死亡，显著延长住院时间
        * "Null" - 无分级

完整格式示例：
[
    {{
      "name": "非心脏手术后心肌损伤",
      "grading": "Null"
    }}
]

注意：
- 如果没有发现术后并发症，请在数组中写明"无术后并发症"，分级为“Null”。
- 请列举你认为的所有并发症，但必须包含在上述的22个之中，禁止输出自定义的并发症
- 常规分级按照{grading}
### 病历资料
"""
    # Concatenate the instruction, extracted complication categories, and output structure
    # to form the complete prompt string.
    return instruction + complications + output_structure

In [5]:
# save as json
import json
prompts_output_path = "comprehensive_prompts_nothink.json"

with open("defination.md", "r") as f:
    text = f.read()
comprehensive_prompts = {}
with open("../data_source/medical_record_example.json", "r") as f:
    data = json.load(f)

for id, medical_record in data.items():
    comprehensive_prompts[id] = get_nothink_comprehensive_prompt(text) + medical_record
with open(prompts_output_path, "w") as f:
    json.dump(comprehensive_prompts, f, ensure_ascii=False, indent=4)

### Having think label (CoT)


In [6]:
def get_comprehensive_prompt(text):
    """
    Generates a comprehensive prompt with Chain of Thought (CoT) for multi-complication assessment.
    
    This function creates a "comprehensive strategy" prompt that requires the AI to evaluate all possible
    post-operative complications simultaneously while providing explicit reasoning. The prompt
    directly formats patient medical records and demands structured thinking before diagnosis.
    
    Key features of the comprehensive strategy prompt:
    - Comprehensive evaluation of all 22 predefined complications at once
    - Mandatory "think" field requiring step-by-step reasoning process
    - Structured JSON output with both reasoning and diagnosis components
    - Direct formatting of medical records into the prompt
    
    Args:
        text (str): The full content of the prompt definition file (e.g., prompt.md),
                    containing the definitions of post-operative complication categories
                    and their grading rules.
    
    Returns:
        str: A complete prompt string ready for use with medical record content appended.
             The prompt includes instructions, complication definitions, output format,
             and placeholders for patient data.
    """
    # Define markers to identify and extract specific sections from the input text
    complication_flag = "## 术后并发症类别"
    grading_flag = "## 常规分级"
    
    # Extract the section detailing all complication categories and their definitions
    complications = text[text.find(complication_flag) + len(complication_flag): text.find(grading_flag)]
    
    # Extract and format the general grading rules, replacing newlines with tabs for better formatting
    grading = text[text.find(grading_flag) + len(grading_flag):].replace("\n", "\t")
    
    # Define the initial instruction establishing the AI's role and comprehensive assessment task
    instruction = '你是一名资深外科医生，你的任务是根据患者的资料，判断患者术后出现哪些并发症，并发症的诊断依据:\n'                
    
    # Define the structured output format requiring both reasoning (think) and diagnosis (complications)
    # This implements Chain of Thought methodology for improved diagnostic accuracy
    output_structure = f"""
### 输出json格式
1. 首先，输出一个think字段，内容为一个数组，每个元素是一个字符串，表示对病例的思考和考虑过程。例如：
    * "患者术后恢复良好，无明显不适"
    * "肌钙蛋白T升高，提示可能存在心肌损伤"
2. 然后，输出一个complications字段，内容为一个数组，每个元素包含：
    * name: 并发症的名称，必须是22个预定义并发症之一
    * grading: 严重程度分级，只能是以下四个选项之一：
        * "轻" - 轻度：无需治疗
        * "中" - 中度：需治疗，无长期影响
        * "重" - 重度：导致器官功能受损或死亡，显著延长住院时间
        * "Null" - 无分级

完整格式示例：
{{
  "think": [
    "患者肾移植术后，根据指南排除急性肾损伤的诊断。",
    "术后首次肌钙蛋白T升高至0.040 ng/mL，符合非心脏手术后心肌损伤的定义。"
  ],
  "complications": [
    {{
      "name": "非心脏手术后心肌损伤",
      "grading": "Null"
    }}
  ]
}}

注意：
- 如果没有发现术后并发症，请在complications数组中写明"无术后并发症"。
- 如发现了术后并发症，则必须包含在上述的22个之中，禁止输出自定义的并发症
- 常规分级按照{grading}
### 病历资料
"""
    # Concatenate the instruction, extracted complication categories, and output structure
    # to form the complete prompt string ready for medical record content
    return instruction + complications + output_structure

In [7]:
# save as json
import json
prompts_output_path = "comprehensive_prompts.json"

with open("defination.md", "r") as f:
    text = f.read()
comprehensive_prompts = {}
with open("../data_source/medical_record_example.json", "r") as f:
    data = json.load(f)

for id, medical_record in data.items():
    comprehensive_prompts[id] = get_comprehensive_prompt(text) + medical_record
with open(prompts_output_path, "w") as f:
    json.dump(comprehensive_prompts, f, ensure_ascii=False, indent=4)

## Modified prompts

After analyzing model errors, we improved prompts with four key changes:

1. **Enhanced Diagnostic Specificity** - Added exclusion criteria (e.g., no acute kidney injury after kidney transplant)
2. **Documentation Clarification** - Handled incomplete records (e.g., paralytic ileus needs GI symptoms)  
3. **Differential Diagnostic Refinement** - Reduced false positives (e.g., distinguish postop bleeding from intraop blood loss)
4. **Anatomical Precision Enhancement** - Expanded definitions (e.g., anastomotic leakage includes external leakage and imaging findings)


In [8]:
# 保存hard_prompts 为json
import json
prompts_output_path = "comprehensive_prompts_modified.json"

with open("defination_modified.md", "r") as f:
    text = f.read()
comprehensive_prompts = {}
with open("../data_source/medical_record_example.json", "r") as f:
    data = json.load(f)

for id, medical_record in data.items():
    comprehensive_prompts[id] = get_comprehensive_prompt(text) + medical_record
with open(prompts_output_path, "w") as f:
    json.dump(comprehensive_prompts, f, ensure_ascii=False, indent=4)

# Targeted strategy

Instead of evaluating all 22 complications at once, this strategy focuses on one complication per call:

- Reduces cognitive load for smaller models
- Each prompt: single complication → True/False + severity  

In [11]:
import re

def get_targeted_prompt(text):
    """
    Generates targeted prompts for single-complication assessment (targeted strategy).
    
    Args:
        text (str): Content from prompt definition file (e.g., prompt_promote.md)
                   containing complication categories and grading rules
    
    Returns:
        dict: Dictionary mapping complication names to their specific prompts
    """
    # Extract complication categories and grading rules from input text
    complication_flag = "## 术后并发症类别"
    grading_flag = "## 常规分级"
    complications = text[text.find(complication_flag) + len(complication_flag): text.find(grading_flag)]
    complication_list = re.split(r'\d+\.\s', complications)[1:]
    grading = text[text.find(grading_flag) + len(grading_flag):].replace("\n", "\t")
    
    # Define instruction template for single-complication assessment
    instruction = '''你是一名资深外科医生，你的任务是根据病例资料，判断患者术后是否出现{complication}，该并发症的诊断依据:\n'''
    
    # Define streamlined output structure for targeted evaluation
    output_structure = """
注意诊断标准必须全部符合（除非标准明确指出符合其中之一）。请输出如下json格式：
{{"think": "...(你的step by step思考过程)", "bool": "True/False(判断患者是否出现并发症，只能输出True和False的其中一个)", "grading" "Null/轻/中/重度（该并发症的严重程度，只能输出一种）"}}
病历资料：
"""
    
    # Generate individual prompts for each complication
    targeted_complications = {}
    for complication in complication_list:
        complication_name, complication_std = complication.split("\n", 1)
        _instruction = instruction.format(complication=complication_name, pt_info="{pt_info}")
        targeted_complications[complication_name] = _instruction + complication_std.replace("常规分级", grading) + output_structure
    
    return targeted_complications

In [12]:
# save targeted prompts as json
import json
with open("defination_modified.md", "r") as f:
    text = f.read()

targeted_complications = get_targeted_prompt(text)

with open("../data_source/medical_record_example.json", "r") as f:
    data = json.load(f)

targeted_prompts = {}
for id, medical_record in data.items():
    targeted_prompts[id] = {key: value + medical_record for key, value in targeted_complications.items()}

with open("targeted_prompts.json", "w") as f:
    json.dump(targeted_prompts, f, ensure_ascii=False, indent=4)

# Training-validation set split

In [1]:
import random
import json

# Generate a list of numbers from 1 to 146
all_ids = list(range(1, 147))

# Shuffle the list of IDs randomly
random.shuffle(all_ids)

# Define the sizes for train and validation sets
train_size = 100
val_size = 46

# Split the shuffled IDs into train and validation sets
train_ids = all_ids[:train_size]
val_ids = all_ids[train_size:train_size + val_size]

# Create a dictionary representing the train/validation split
split_data = {
    "train": train_ids,
    "val": val_ids
}

with open("../data_source/center1/train_val.json", "w", encoding="utf-8") as f:
    json.dump(split_data, f, ensure_ascii=False, indent=4)

Train set size: 100
Validation set size: 46
